In [1]:
import sys
from pathlib import Path

import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns
sns.set_theme(style="whitegrid", palette="viridis")

sys.path.append(str(Path().resolve().parent / "src"))
from utils import load_all_csvs
from sanitize import sanitize_all

In [ ]:
dfs = load_all_csvs(Path.cwd().parent / "data")
dfs = sanitize_all(dfs)

In [ ]:
dfs['transactional'].columns

In [ ]:
# Garante que a coluna ZCTA existe (através do Lab Id)
dfs['transactional'] = dfs['transactional'].merge(
    dfs['geocode'][['Lab Id', 'Zipcode']],
    on='Lab Id',
    how='left'
)
if 'ZCTA' not in dfs['transactional'].columns:
    dfs['transactional']['ZCTA'] = dfs['transactional']['Zipcode'].astype(str).str.split('.').str[0].str.zfill(5)

# 1. Diversidade de exames por ZCTA
diversidade_exames = (
    dfs['transactional']
    .groupby('ZCTA')['CodItem']
    .nunique()
    .reset_index()
    .rename(columns={'CodItem': 'num_exames_unicos'})
)

# 2. Volume total de exames por ZCTA
volume_exames = (
    dfs['transactional']
    .groupby('ZCTA')
    .size()
    .reset_index(name='num_exames_totais')
)

# 3. Merge para DataFrame final
h5a_df = pd.merge(diversidade_exames, volume_exames, on='ZCTA')
h5a_df['proporcao_diversidade'] = h5a_df['num_exames_unicos'] / h5a_df['num_exames_totais']

# 4. Prepara Top 10 ZCTAs por diversidade
top_diversidade = h5a_df.sort_values(by='num_exames_unicos', ascending=False).head(10)

# 5. Plota os dois gráficos lado a lado
fig, axes = plt.subplots(1, 2, figsize=(16, 5))

# Scatterplot
sns.scatterplot(data=h5a_df, x='num_exames_totais', y='num_exames_unicos', ax=axes[0])
axes[0].set_title('Diversidade vs. Volume de Exames por ZCTA')
axes[0].set_xlabel('Volume Total de Exames')
axes[0].set_ylabel('Tipos Diferentes de Exames')

# Barplot Top 10
sns.barplot(data=top_diversidade, x='num_exames_unicos', y='ZCTA', hue='ZCTA', ax=axes[1], palette='mako')
axes[1].set_title('Top 10 ZCTAs com Maior Diversidade de Exames')
axes[1].set_xlabel('Tipos Diferentes de Exames')
axes[1].set_ylabel('ZCTA')

plt.tight_layout()
plt.show()

### 📊 H5a – Diversidade de Exames como Indicador de Fidelização

A análise da diversidade de exames por ZCTA revela que:

- Existe uma **correlação clara** entre o volume total de exames realizados e a diversidade de tipos distintos de exames.
- ZCTAs como `98008`, `94565`, `30096` e `95823` se destacam não apenas pelo alto volume, mas também por apresentarem **grande variedade de procedimentos**, sugerindo uma **demanda ampla e continuada** dos serviços laboratoriais.
- A presença de muitos tipos diferentes de exames realizados pode indicar que os pacientes retornam para diversas especialidades ou utilizam o laboratório como **único ponto de atendimento**, o que é um **sinal de fidelização**.

> **Conclusão**: A hipótese H5a é **suportada pelos dados** — ZCTAs com maior diversidade de exames tendem a representar regiões com **alto grau de fidelidade dos pacientes**, refletindo maior retenção e engajamento da base de clientes.


In [ ]:
paciente_freq = dfs['transactional'].groupby('Patient Id').size().reset_index(name='num_visitas')
paciente_freq